### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [ ]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [13]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [14]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [15]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [16]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [17]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [18]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [19]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [20]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several compelling reasons to consider using AutoGen in your AI Agent project:

1. **Enhanced Collaboration**: AutoGen enables multiple AI agents to communicate and collaborate effectively, allowing for better task coordination and feedback loops. This feature can lead to faster development cycles and reduced workload for developers.

2. **Customization and Versatility**: The framework allows for highly customizable and conversable agents. You can adapt the agents to suit specific project needs, including incorporating human inputs and leveraging different combinations of Language Learning Models (LLMs).

3. **Integration with Tools and APIs**: AutoGen supports integration with various tools and external systems, which enhances the functionality of your AI agents. This allows for comprehensive solutions that can execute complex tasks autonomously.

4. **Open-Source Advantage**: Being open-source means that AutoGen not only reduces costs but also allows you to tap into a community of developers for support, enhancements, and shared developments.

5. **Simplification of Workflow Management**: The framework simplifies the orchestration, optimization, and automation of LLM workflows, making it easier to develop and maintain applications that harness the power of advanced AI.

6. **Scalability**: AutoGen is designed to improve scalability in AI projects, enabling you to scale your operations as needed without compromising on performance or speed.

In summary, AutoGen offers a range of benefits that can enhance the effectiveness, efficiency, and functionality of your AI Agent project, making it a strong choice for various applications.

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your AI Agent project:

1. **Complex Documentation**: Many users find AutoGen's documentation difficult to navigate, with insufficient examples to aid understanding. This may hinder implementation and lead to increased development time.

2. **Limited Features**: Compared to competitors, AutoGen may lack certain features, such as a visual builder or no-code options, which could make it less accessible for teams without advanced programming skills.

3. **Learning Curve**: The framework has a steeper learning curve, which could result in longer onboarding times for team members and potentially delay project timelines.

4. **Cost**: For large-scale projects, AutoGen can be expensive, which might not be justifiable for every organization, particularly startups or smaller teams.

5. **Risk of Over-Engineering**: The complexity of the framework can lead to over-engineered solutions that are not necessary for simpler tasks, causing inefficiencies in system design.

6. **Not Ideal for All Project Types**: AutoGen excels in scenarios with multiple AI agents collaborating dynamically but may not be the best choice for projects that do not require this level of coordination.

In summary, while AutoGen has powerful capabilities, its complexity, cost, and steep learning curve may make it a less appealing option for some projects. 

TERMINATE



## Decision:

Based on the provided pros and cons of using AutoGen for the AI Agent project, I recommend using AutoGen. 

The rationale for this decision is that the advantages significantly outweigh the disadvantages. The enhanced collaboration between AI agents, the flexibility for customization, and the ability to integrate with various tools present strong benefits that can lead to better project outcomes. While there are concerns about documentation complexity and a potential learning curve, the open-source nature and community support can help mitigate these issues over time.

The capacity for scalability and workflow management provided by AutoGen can greatly enhance the efficiency and effectiveness of the project, making it a strong candidate overall.

TERMINATE

In [21]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [22]:
await host.stop()